In [186]:
import pandas as pd

In [217]:
df_perf = pd.read_excel('implementation_assessment/wa2_keyword_performance.xlsx')

In [218]:
df_prod = pd.read_excel('implementation_assessment/wa2_product_database.xlsx')

In [219]:
df_bid = pd.read_excel('implementation_assessment/wa2_keyword_bid.xlsx')

In [220]:
# Find the most recent time and the previous 30 days.
# Filter down the record within the time period
df_mostRecentDate = df_perf.groupby(['keyword_id']).max()
df_mostRecentDate['minus30'] = df_mostRecentDate['date'] - pd.DateOffset(30, 'D')
df_mostRecentDate = df_mostRecentDate.reset_index()[['keyword_id','date','minus30']]
df_bothDate = pd.merge(df_perf, df_mostRecentDate, on="keyword_id", how="inner")
df_bothDate = df_bothDate[(df_bothDate["date_x"] >= df_bothDate["minus30"])]

In [221]:
# Aggregate into the keyword_id-base record.
# The metrics related to ACoS are using sum() for later calculation
# Rest of text data are using the fist
# TODO: using mostFrequency()
df_merge_product = pd.merge(df_bothDate, df_prod, on="short_id")
df_merge_product = df_merge_product.drop(['date_x', 'date_y', 'minus30'], axis=1)
df_asos = df_merge_product.groupby(['keyword_id']).agg({
    'Breakeven Cost' : 'sum', 
    'Breakeven ACoS': 'sum', 
    'cost': 'sum',
    'sales': 'sum',
    'units_ordered': 'sum',
    'impressions': 'sum',
    'clicks': 'sum',
    'conversions': 'sum',
    'top_search_multiplier' : 'mean',
    'match_type' : 'first',
    'campaign_id' : 'first',
    'campaign_name' : 'first',
    'short_id' : 'first',
    'keyword_text': 'first'
    })

In [222]:
# Merge the bid data of product to calcuate the current effective bidding
df_merge_bid = pd.merge(df_asos, df_bid, on="keyword_id")
df_merge_bid['effective_bid'] = df_merge_bid['bid'] * df_merge_bid['top_search_multiplier']

In [223]:
# Apply the optimaliization rules, using different lambda function to check the acos, breakeven scos, units,...
# Adding a column to identify which rule has been applied.
bidOp = df_merge_bid.copy()
bidOp = bidOp.fillna(0)
def optimalBids(cost, sales, breakevenASOS, breakevenCost, units):
    if sales==0 and cost>(breakevenCost*1.25):
        return 0.75
    if sales==0:
        return 1
    if cost/sales > (breakevenASOS+0.1):
        return 0.75
    elif cost/sales < (breakevenASOS-0.05) and (units < 2):
        return 1.15
    return 1
def optimalRules(sales, ratio):
    if ratio == 0.75 and sales == 0:
        return "Rule 3"
    if ratio == 0.75:
        return "Rule 1"
    if ratio == 1.15:
        return "Rule 2"
    return "No rule applied"
    
def optimalBidsState(currentState, cost, sales, breakevenASOS, breakevenCost):
    if sales==0 and cost>breakevenCost*2.5:
        return "paused"
    if sales==0:
        return currentState
    if cost/sales > 2.75 * breakevenASOS:
        return "paused"
    return currentState
def optimalRulesState(sales, state, rule):
    if sales == 0 and state == "paused":
        return "Rule 5"
    if state == "paused":
        return "Rule 4"
    return rule
bidOp['new_effective_bid_ratio'] = bidOp.apply(lambda x:optimalBids(x['cost'],x['sales'],x['Breakeven ACoS'],x['Breakeven Cost'],x['units_ordered']), axis=1)
bidOp['new_effective_bid'] = bidOp['effective_bid'] * bidOp['new_effective_bid_ratio']
bidOp['proposed_state'] = bidOp.apply(lambda x:optimalBidsState(x['state'], x['cost'],x['sales'],x['Breakeven ACoS'],x['Breakeven Cost']), axis=1)

bidOp['rule_applied'] = bidOp.apply(lambda x:optimalRules(x['sales'],x['new_effective_bid_ratio']), axis=1)
bidOp['rule_applied'] = bidOp.apply(lambda x:optimalRulesState(x['sales'],x['new_effective_bid_ratio'],x['rule_applied']), axis=1)

In [224]:
# Apply additional potimal rules
bidOp2 = bidOp.copy()
def optimalTopword(name, bid):
    if "Top Keywords" in name:
        return 10
    elif bid > 8:
        return 8
    elif bid < 0.2:
        return 0.2
    return bid
bidOp2["new_effective_bid"] = bidOp2.apply(lambda x:optimalTopword(x['campaign_name'], x["new_effective_bid"]), axis=1)
bidOp2["new_bid"] = bidOp2["new_effective_bid"] / bidOp2["top_search_multiplier"]

In [225]:
# Organize the column
bidOp2['Breakeven Cost+10'] = bidOp2['Breakeven Cost'] + 10
bidOp2['Breakeven Cost-10'] = bidOp2['Breakeven Cost'] - 5
bidOp2['Breakeven ACoSx2.75'] = bidOp2['Breakeven ACoS'] * 2.75
bidOp2['Breakeven Costx1.25'] = bidOp2['Breakeven Cost'] * 1.25
bidOp2['Breakeven Costx2.5'] = bidOp2['Breakeven Cost'] * 2.5
bidOp2.drop(['new_effective_bid_ratio'], axis=1)
bidOp2['ACoS'] = bidOp2['cost'] / bidOp2['sales']
bidOp2.rename(columns={'cost':'Cost (aggregated over desired time period)',
                      'sales':'Sales (aggregated over desired time period)',
                       'units_ordered':'Units Sold (aggregated over desired time period)',
                       'conversions':'Conversions (aggregated over desired time period)',
                       'impressions': 'Impressions (aggregated over desired time period)',
                       'clicks': 'Clicks (aggregated over desired time period)',
                       'bid': 'Current bid',
                       'rule_applied':'Bid Optimization Rule(s) Applied',
                       'new_effective_bid':'Final Proposed Effective Bid',
                       'new_bid':'Final Proposed (Raw) Bid',
                       'proposed_state':'Final Proposed State'
                      })

,keyword_id,Breakeven Cost,Breakeven ACoS,Cost (aggregated over desired time period),Sales (aggregated over desired time period),Units Sold (aggregated over desired time period),Impressions (aggregated over desired time period),Clicks (aggregated over desired time period),Conversions (aggregated over desired time period),top_search_multiplier,...,Final Proposed Effective Bid,Final Proposed State,Bid Optimization Rule(s) Applied,Final Proposed (Raw) Bid,Breakeven Cost+10,Breakeven Cost-10,Breakeven ACoSx2.75,Breakeven Costx1.25,Breakeven Costx2.5,ACoS
0,257135855104,240.8130,4.544499,325.60,2303.98,47,798,51,30,10.0,...,10.0,enabled,No rule applied,1.0,250.8130,235.8130,12.497372,301.01625,602.0325,0.141321
1,665395867002,388.8900,14.454349,157.70,2291.90,82,15728,152,78,2.5,...,8.0,enabled,No rule applied,3.2,398.8900,383.8900,39.749460,486.11250,972.2250,0.068808
2,976622689600,101.2000,6.764300,422.44,1311.39,69,21786,240,66,0.0,...,0.2,enabled,No rule applied,inf,111.2000,96.2000,18.601825,126.50000,253.0000,0.322131
3,1915220456682,408.8350,7.173802,69.40,455.92,8,664,13,8,10.0,...,10.0,enabled,No rule applied,1.0,418.8350,403.8350,19.727957,511.04375,1022.0875,0.152220
4,3642387958158,359.7748,6.312946,637.31,2598.93,47,4203,102,40,2.5,...,8.0,enabled,No rule applied,3.2,369.7748,354.7748,17.360602,449.71850,899.4370,0.245220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,144335981199493504,266.6856,5.334779,93.96,252.95,5,38597,33,5,0.0,...,0.2,enabled,No rule applied,inf,276.6856,261.6856,14.670642,333.35700,666.7140,0.371457
496,144341912666040000,347.5300,8.690423,93.80,79.98,2,17479,40,2,0.0,...,0.2,enabled,No rule applied,inf,357.5300,342.5300,23.898662,434.41250,868.8250,1.172793
497,144345874365447296,346.2500,6.926385,454.92,1798.68,42,37443,164,41,0.0,...,0.2,enabled,No rule applied,inf,356.2500,341.2500,19.047560,432.81250,865.6250,0.252919
498,144356443740833696,290.8500,5.818164,70.29,349.93,7,14560,42,6,0.0,...,0.2,enabled,No rule applied,inf,300.8500,285.8500,15.999950,363.56250,727.1250,0.200869


In [216]:
bidOp2.to_excel('implementation_assessment/ChenYang_Yu_implementation_assessment.xlsx')